In [ ]:
import pandas as pd
import numpy as np

df1 = pd.read_csv("Rejected Loans.csv",low_memory=False)
df2 = pd.read_csv("Accepted Loans.csv",low_memory=False)

dfloan = df2[['loan_amnt','title','dti','addr_state','emp_length','policy_code']]
dfloan = dfloan.dropna(how='any',axis=0) #remove NaN's

#convert the employment data into numerics 
emp_map = {'emp_length':{"< 1 year":0.5,"1 year":1,"2 years":2, "3 years":3,"4 years":4,
                         "5 years":5,"6 years":6, "7 years":7,"8 years":8,"9 years":9,
                         "10 years":10,"10+ years":11}} 
dfloan.replace(emp_map, inplace=True)

#adding the variable of whether they are apporved 
dfloan['Approved']=pd.Series(1, dfloan.index)

dfrej = df1[['Amount Requested','Loan Title','Debt-To-Income Ratio','State',
            'Employment Length','Policy Code']]
dfrej = dfrej.rename(columns = {'Amount Requested':'loan_amnt','Loan Title':'title', 
                                'Debt-To-Income Ratio':'dti','State':"addr_state", 
                                'Employment Length':'emp_length',
                                'Policy Code':'policy_code'})
dfrej = dfrej.dropna() #remove NaN's
dfrej.replace(emp_map, inplace=True)
#converting dti from strings into floats 
dfrej['dti']= dfrej['dti'].replace('%', '',regex=True).astype('float')
dfrej['Approved']=pd.Series(0,dfrej.index)

#joining two data sets 
df = pd.concat([dfloan,dfrej]) 

#split predictive variables from predicted output "Approved"
dfs = np.split(df,[6],axis=1)
df = dfs[0] 

#predicted outputs needs to be integers 
dfy = np.ravel(dfs[1]).astype('int')

#converting categorical data into indicators 
df = pd.concat([df, pd.get_dummies(df['title'], prefix = '')], axis=1) 
df = pd.concat([df, pd.get_dummies(df['addr_state'], prefix = '')], axis=1) 
df = df.drop('title',1) 
df = df.drop('addr_state',1) 

#After concatenation, convert the datatype again 
df["loan_amnt"] = df["loan_amnt"].astype('int') 
df["dti"] = df["dti"].astype('int') 
df['emp_length'] = df['emp_length'].astype('float') 
df['policy_code'] = df['policy_code'].astype('int')

#Spliting the data into training and test sets 
x_train, x_test, y_train, y_test = train_test_split(df,dfy,test_size=0.35, random_state=0) 

#Training the regression model 
logreg = LogisticRegression() 
logreg.fit(x_train, y_train) 
predictions = logreg.predict(x_test) 

#obtain the scores to measure if it's 
print metrics.accuracy_score(y_test, predictions)

#metrics for classificatioin 
print metrics.r2_score(y_test, predictions) 

#metrics for regression 
print metrics.classification_report(y_test, predictions)

min_loan_amnt = df['loan_amnt'].min #2000 

def predict_max(x): 
    #input is an array of covariates that excluding loan amount 
    amnt = pd.DataFrame(data = {"Approved": [1000]}) 
    #I picked 10000 as an arbitrary baseline because its smaller than 16000 
    x = pd.concat([amnt,x],axis=1) 
    for i in range(len(df.index)): 
        y = logreg.predict(x) 
        x.at[0,"Approved"] += 10 
        if y == 0: 
            max = x.at[0,"Approved"] 
            break 
    return max

x = df.iloc[-1] 
print "The recommended maximum loan amount for an applicant with the following details is $",predict_max(x)
print x